In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("banco.db")

In [3]:
cur = con.cursor()

In [4]:
# result = cur.execute("SELECT name FROM sqlite_master")
# exists = any("movie" in record for record in result)

In [4]:
# if not exists:
#     cur.execute("CREATE TABLE movie(title, year, score)")
cur.execute("CREATE TABLE IF NOT EXISTS movie(title, year, score)")

In [5]:
res = cur.execute("SELECT name FROM sqlite_master")
# res.fetchone()
res.fetchall()

[('movie',)]

In [6]:
cur.execute(
    """
INSERT INTO movie(title, year, score)
VALUES ('The Shawshank Redemption', 1994, 9.3)
            """
)

In [7]:
cur.execute("SELECT * from movie").fetchall()

[('The Shawshank Redemption', 1994, 9.3)]

In [8]:
con.commit()

In [9]:
result = cur.execute("SELECT * FROM movie")
result.fetchall()

[('The Shawshank Redemption', 1994, 9.3)]

In [10]:
cur.execute(
    """
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
"""
)
con.commit()

In [11]:
result = cur.execute("SELECT * FROM movie")
result.fetchall()

[('The Shawshank Redemption', 1994, 9.3),
 ('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5)]

In [12]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()

In [13]:
result = cur.execute("SELECT * FROM movie")
result.fetchall()

[('The Shawshank Redemption', 1994, 9.3),
 ('Monty Python and the Holy Grail', 1975, 8.2),
 ('And Now for Something Completely Different', 1971, 7.5),
 ('Monty Python Live at the Hollywood Bowl', 1982, 7.9),
 ("Monty Python's The Meaning of Life", 1983, 7.5),
 ("Monty Python's Life of Brian", 1979, 8.0)]

In [14]:
# recuperando todos os dados com score maior que 8
target = 8.0
records = []
for result in cur.execute("SELECT * FROM movie"):
    if result[-1] > target:
        records.append(result)

records

[('The Shawshank Redemption', 1994, 9.3),
 ('Monty Python and the Holy Grail', 1975, 8.2)]

In [15]:
result = cur.execute("SELECT * FROM movie WHERE score > 8.0")
result.fetchall()

[('The Shawshank Redemption', 1994, 9.3),
 ('Monty Python and the Holy Grail', 1975, 8.2)]

In [16]:
cur.execute("UPDATE movie SET score = 9.0 WHERE title = 'The Shawshank Redemption'")

In [17]:
con.commit()

In [18]:
cur.execute("SELECT * FROM movie WHERE title = 'The Shawshank Redemption'").fetchall()

[('The Shawshank Redemption', 1994, 9.0)]

In [19]:
con.close()

In [20]:
# É esperado que tenhamos um erro, pois a conexão foi interrompida.
cur.execute("SELECT * FROM movie").fetchall()

ProgrammingError: Cannot operate on a closed database.

# Utilizando o pandas para trabalhar com o banco de dados

In [21]:
import pandas as pd

In [38]:
bd_carros = pd.read_csv("bd.csv")  # , parse_dates=["Report Received Date"])
bd_carros["Potentially Affected"] = (
    bd_carros["Potentially Affected"].fillna(0).astype("int")
)
bd_carros.head()

,Report Received Date,Manufacturer,Component,Recall Type,Potentially Affected
0,01/06/2023,Triple E Recreational Vehicles,ELECTRICAL SYSTEM,Vehicle,341
1,01/05/2023,"Volvo Car USA, LLC",STEERING,Vehicle,74
2,12/29/2022,"Volkswagen Group of America, Inc.",ELECTRICAL SYSTEM,Vehicle,1042
3,12/29/2022,Indian Motorcycle Company,STRUCTURE,Vehicle,4653
4,12/29/2022,Hendrickson USA. L.L.C.,"SERVICE BRAKES, AIR",Equipment,12


In [43]:
bd_carros.columns = [r.lower().replace(" ", "_") for r in bd_carros.columns]

In [44]:
bd_carros.head()

,report_received_date,manufacturer,component,recall_type,potentially_affected
0,01/06/2023,Triple E Recreational Vehicles,ELECTRICAL SYSTEM,Vehicle,341
1,01/05/2023,"Volvo Car USA, LLC",STEERING,Vehicle,74
2,12/29/2022,"Volkswagen Group of America, Inc.",ELECTRICAL SYSTEM,Vehicle,1042
3,12/29/2022,Indian Motorcycle Company,STRUCTURE,Vehicle,4653
4,12/29/2022,Hendrickson USA. L.L.C.,"SERVICE BRAKES, AIR",Equipment,12


In [39]:
bd_carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26590 entries, 0 to 26589
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Report Received Date  26590 non-null  object
 1   Manufacturer          26590 non-null  object
 2   Component             26590 non-null  object
 3   Recall Type           26590 non-null  object
 4   Potentially Affected  26590 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


In [49]:
con = sqlite3.connect("banco.db")
cur = con.cursor()

In [50]:
cur.execute(
    """
    CREATE TABLE IF NOT EXISTS cars(
        report_received_date,
        manufacturer,
        component,
        recall_type,
        potentially_affected
        )
    """
)
data = bd_carros.head().to_records(index=False).tolist()
cur.executemany(
    """
    INSERT INTO cars VALUES(?, ?, ?, ?, ?)
    """,
    data,
)
con.commit()

In [53]:
bd_carros.report_received_date = bd_carros.report_received_date.apply(pd.to_datetime)

In [54]:
bd_carros.to_sql("cars2", con, index=False)

26590

In [55]:
bd_carros.shape

(26590, 5)

In [58]:
pd.read_sql(
    """
    SELECT *
    FROM cars2
    WHERE potentially_affected >= 20000
    """,
    con,
)

,report_received_date,manufacturer,component,recall_type,potentially_affected
0,2022-12-23 00:00:00,"WARN Industries, Inc.",ELECTRICAL SYSTEM,Equipment,121187
1,2022-12-23 00:00:00,"Mercedes-Benz USA, LLC","FUEL SYSTEM, DIESEL",Vehicle,323963
2,2022-12-23 00:00:00,Ford Motor Company,SEAT BELTS,Vehicle,101001
3,2022-12-23 00:00:00,"Mercedes-Benz USA, LLC",VISIBILITY,Vehicle,123696
4,2022-12-21 00:00:00,"Volkswagen Group of America, Inc.",AIR BAGS,Vehicle,37558
...,...,...,...,...,...
4216,1966-10-10 00:00:00,"Volkswagen Group of America, Inc.","SERVICE BRAKES, HYDRAULIC",Vehicle,64596
4217,1966-10-06 00:00:00,Ford Motor Company,SEAT BELTS,Vehicle,65000
4218,1966-01-19 00:00:00,"General Motors, LLC",STEERING,Vehicle,138878
4219,1966-01-19 00:00:00,"General Motors, LLC",STEERING,Vehicle,70644
